# Gaussian LIBOR Market Model - Discount Bond Prices

## Problem Setup

We consider a Gaussian LIBOR Market Model with forward LIBOR rates $f_i(t)$ with $\Delta T = 1$ and $N = 10$.

The final time-point in this model is $T_{10} = 10$ years.

The initial term-structure of interest rates is equal to $f_i(0) = 0.03 = 3\%$ for all $i = 0, ..., 9$.

## Mathematical Formulation

### Forward LIBOR Definition

By definition of discretely-compounded forward LIBOR:

$$1 + \Delta T \cdot f_i(0) = \frac{P(0,T_i)}{P(0,T_{i+1})}$$

where $P(0,T_k)$ is the discount bond price at $t=0$ maturing at $T_k$.

### Computing Discount Bonds

With $\Delta T = 1$ and $f_i(0) = 0.03$:

$$1 + \Delta T \cdot f_i(0) = 1 + 1 \cdot 0.03 = 1.03$$

Therefore:

$$P(0,T_{i+1}) = \frac{P(0,T_i)}{1.03}$$

With initial condition $P(0,T_0) = P(0,0) = 1$, we obtain:

$$P(0,T_k) = \frac{1}{(1.03)^k}, \quad k = 1, ..., 9$$

In [ ]:
## Results: Discount Bond Prices at t=0

| Maturity T | P(0,T) |
|------------|-----------|
| 1 | 0.970873786 |
| 2 | 0.942595909 |
| 3 | 0.915141660 |
| 4 | 0.888486854 |
| 5 | 0.862608597 |
| 6 | 0.837484463 |
| 7 | 0.813091705 |
| 8 | 0.789409425 |
| 9 | 0.766416917 |

# Monte Carlo Simulation - Gaussian LIBOR Market Model

## Model and Analytic Discount Factors

**Parameters:**
- Tenor: $\Delta T = 1$, dates $T_i = i$ for $i = 0, ..., 10$
- Forwards at $t=0$: $f_i(0) = 0.03$ for $i = 0, ..., 9$
- Volatilities: $\sigma_i \equiv 0.01$

**Forward LIBOR Definition** (discrete compounding, $\Delta T = 1$):

$$1 + f_i(t) = \frac{D_i(t)}{D_{i+1}(t)} \quad \Longrightarrow \quad D_{i+1}(t) = \frac{D_i(t)}{1 + f_i(t)}$$

At $t=0$, all $f_i(0) = 0.03$, so:

$$D_0(0) = 1, \quad D_k(0) = \frac{1}{(1.03)^k}, \quad k = 1, ..., 10$$

These are the **analytic discount factors** we compare against.

## SDE for LIBORs under Terminal Measure $\mathbb{Q}^{10}$

Under the final measure $\mathbb{Q}^{10}$ with numéraire $D_{10}(t)$:

$$df_i(t) = -\sum_{k=i+1}^{9} \frac{\Delta T \cdot \sigma_k(t)}{1 + \Delta T \cdot f_k(t)} \sigma_i(t) \, dt + \sigma_i(t) \, dW_t^{\mathbb{Q}^{10}}$$

for $i = 0, ..., 9$.

With $\Delta T = 1$ and $\sigma_i(t) \equiv \sigma = 0.01$:

$$df_i(t) = -\sigma^2 \sum_{k=i+1}^{9} \frac{1}{1 + f_k(t)} dt + \sigma \, dW_t, \quad i = 0, ..., 9$$

The last forward $f_9$ has **zero drift** (sum is empty).

## Pricing Bonds under Numéraire $D_{10}$

We want $D_k(0)$ via MC for $k = 1, ..., 9$.

**General numéraire pricing:**

$$D_k(0) = D_{10}(0) \cdot \mathbb{E}^{\mathbb{Q}^{10}}\left[\frac{D_k(T_k)}{D_{10}(T_k)}\right]$$

At maturity $T_k$: $D_k(T_k) = 1$. So:

$$D_k(0) = D_{10}(0) \cdot \mathbb{E}^{\mathbb{Q}^{10}}\left[\frac{1}{D_{10}(T_k)}\right]$$

**Express $1/D_{10}(T_k)$ in terms of LIBORs at $T_k$:**

Using the forward LIBOR relationship:

$$D_{i+1}(T_k) = \frac{D_i(T_k)}{1 + f_i(T_k)} \quad \Longrightarrow \quad D_{10}(T_k) = \frac{D_k(T_k)}{\prod_{j=k}^{9}(1 + f_j(T_k))}$$

Since $D_k(T_k) = 1$:

$$\frac{1}{D_{10}(T_k)} = \prod_{j=k}^{9}(1 + f_j(T_k))$$

Therefore:

$$D_k(0) = D_{10}(0) \cdot \mathbb{E}^{\mathbb{Q}^{10}}\left[\prod_{j=k}^{9}(1 + f_j(T_k))\right]$$

## Monte Carlo Estimator

With $M$ paths:

$$\widehat{D}_k(0) = D_{10}(0) \cdot \frac{1}{M} \sum_{m=1}^{M} \prod_{j=k}^{9}(1 + f_j^{(m)}(T_k))$$

**Standard Error:**

$$\text{StdErr}_k = D_{10}(0) \cdot \frac{\text{Std}(\Pi_k^{(m)})}{\sqrt{M}}, \quad \Pi_k^{(m)} := \prod_{j=k}^{9}(1 + f_j^{(m)}(T_k))$$

**Z-score:**

$$Z_k = \frac{\widehat{D}_k(0) - D_k(0)}{\text{StdErr}_k}$$

We expect all $Z_k \in [-2, 2]$ for a valid simulation.

## Monte Carlo Simulation Results (100,000 paths) - Q^10 Measure

Simulation parameters:
- Time discretization: dt = 0.01 years
- Number of paths: M = 100,000
- Volatility: σ = 0.01 (1%) for all forwards  
- Numéraire: D_10(t) (terminal measure Q^10)
- Forward LIBOR: f_i(0) = 0.03 = 3% for all i = 0...9
- Final time: T_10 = 10 years
- Drift under Q^10: $\mu_i^{10}(t) = -\sigma^2 \sum_{j=i+1}^{9} \frac{1}{1 + f_j(t)}$

| Maturity T | Analytic P(0,T) | Monte Carlo | Std Error | Z-score |
|------------|-----------------|-------------|-----------|---------|
| 1 | 0.970873786 | 0.970445923 | 2.678e-04 | -1.60 |
| 2 | 0.942595909 | 0.942146407 | 3.287e-04 | -1.37 |
| 3 | 0.915141659 | 0.915099863 | 3.425e-04 | -0.12 |
| 4 | 0.888487048 | 0.888098985 | 3.278e-04 | -1.18 |
| 5 | 0.862608784 | 0.862220771 | 2.966e-04 | -1.31 |
| 6 | 0.837484257 | 0.837169943 | 2.523e-04 | -1.25 |
| 7 | 0.813091511 | 0.812920139 | 1.979e-04 | -0.87 |
| 8 | 0.789409234 | 0.789315481 | 1.369e-04 | -0.68 |
| 9 | 0.766416732 | 0.766352813 | 7.036e-05 | -0.91 |

**Result:** All Z-scores are within [-2, 2], confirming the Monte Carlo simulation correctly reproduces the analytical discount bond prices. The simulation successfully validates the Gaussian LIBOR Market Model implementation under the Q^10 terminal measure with 100,000 paths providing tighter standard errors than the 10,000 path simulation.

# Gaussian Swaption Pricing

## Setup

**Tenor:** $\Delta T = 1$

**Dates:** $T_0 = 0, T_1, ..., T_{10}$ (years)

**Flat initial curve:** $f_i(0) = 0.03 \Rightarrow D(0,T_k) = (1/1.03)^k$

**Payer swaptions:**
- Option maturity $T_i$, $i = 1, ..., 9$
- Underlying swap: fixed vs float on $[T_i, T_{10}]$
- Strikes $K \in \{1\%, 2\%, 3\%, 4\%, 5\%\}$
- Notional $L = 100$
- Gaussian volatility of par swap rate: $\sigma = 0.01$

## Gaussian Formula

For a payer swaption at time $t$ on a swap starting at $T_0$ with maturity $T_N$:

$$V_{\text{pay},t} = L \left(\sum_{k=1}^{N} D_{T_k,t}\right) \left[(y_{T_0 T_N,t} - K) \mathcal{N}(d) + \sigma\sqrt{T_0-t} \, \varphi(d)\right]$$

where

$$d = \frac{y_{T_0 T_N,t} - K}{\sigma\sqrt{T_0-t}}$$

Here we need $t = 0$, $T_0 = T_i$, $T_N = T_{10}$.

## Ingredients at $t = 0$

### Discount Factors

$$D_k(0) = \frac{1}{(1.03)^k}, \quad k = 0, ..., 10$$

### Swap Annuity for Swap $[T_i, T_{10}]$

Payments at $T_{i+1}, ..., T_{10}$:

$$A_i(0) = \sum_{k=i+1}^{10} D_k(0)$$

### Par Swap Rate for Swap $[T_i, T_{10}]$

$$y_{i,10}(0) = \frac{D_i(0) - D_{10}(0)}{A_i(0)}$$

### Gaussian Payer Swaption Price at $t = 0$

For strike $K$ and maturity $T_i$ (so $T_0 - t = i$):

$$\tau_i = T_i - 0 = i, \quad d = \frac{y_{i,10}(0) - K}{\sigma\sqrt{\tau_i}}$$

$$V_{\text{pay}}(0; T_i, K) = L \cdot A_i(0) \left[(y_{i,10}(0) - K) \mathcal{N}(d) + \sigma\sqrt{\tau_i} \, \varphi(d)\right]$$

## Gaussian Payer Swaption Prices

**Model Parameters:**
- Notional: $L = 100$
- Initial forward rate: $f_i(0) = 0.03 = 3\%$ for all $i$
- Gaussian volatility: $\sigma = 0.01 = 1\%$
- Tenor: $\Delta T = 1$
- Final maturity: $T_{10} = 10$ years

### Results Table

| Maturity | K = 1% | K = 2% | K = 3% | K = 4% | K = 5% |
|----------|--------|--------|--------|--------|--------|
| T = 1 | 15.1828 | 8.1891 | 3.0157 | 0.6298 | 0.0642 |
| T = 2 | 13.5660 | 7.9377 | 3.7331 | 1.3210 | 0.3325 |
| T = 3 | 12.0107 | 7.4295 | 3.9397 | 1.7279 | 0.6075 |
| T = 4 | 10.4282 | 6.7171 | 3.8403 | 1.9040 | 0.8020 |
| T = 5 | 8.7973 | 5.8460 | 3.5241 | 1.8955 | 0.8963 |
| T = 6 | 7.1163 | 4.8486 | 3.0421 | 1.7356 | 0.8903 |
| T = 7 | 5.3898 | 3.7489 | 2.4276 | 1.4490 | 0.7900 |
| T = 8 | 3.6241 | 2.5651 | 1.7044 | 1.0546 | 0.6031 |
| T = 9 | 1.8255 | 1.3116 | 0.8906 | 0.5675 | 0.3373 |

**Observations:**
- Swaption values decrease as option maturity increases (for fixed strike)
- For strikes below the par swap rate (~3%), prices are higher
- For strikes above the par swap rate, prices decrease rapidly
- The at-the-money (K ≈ 3%) swaptions show moderate pricing across maturities

# Monte Carlo Swaption Pricing in the LMM

## Objective

Use the Monte Carlo simulation of the Gaussian LIBOR Market Model to calculate the value of all swaptions. Compare the accuracy of the Gaussian closed-form swaption prices as an approximation to the full LMM Monte Carlo swaption values using Z-scores.

## Mathematical Framework

### 1. Model Dynamics under Terminal Measure $\mathbb{Q}^{10}$

**Forward LIBOR SDE:**

$$df_i(t) = \mu_i^{\mathbb{Q}^{10}}(t,f(t)) \, dt + \sigma \, dW_t, \quad i=0,\dots,9$$

**Drift under $\mathbb{Q}^{10}$:**

$$\mu_i^{\mathbb{Q}^{10}}(t,f(t)) = -\sigma^2 \sum_{k=i+1}^{9} \frac{1}{1+f_k(t)}$$

For $i=9$, the sum is empty, so $\mu_9^{\mathbb{Q}^{10}} = 0$.

### 2. Pathwise Swaption Payoff at $T_i$

For each path $m$ and option maturity $T_i$:

**Step 1:** Build discount factors at $T_i$ from forwards:

$$D_i^{(m)}(T_i) = 1$$

$$D_{k+1}^{(m)}(T_i) = \frac{D_k^{(m)}(T_i)}{1 + f_k^{(m)}(T_i)}, \quad k=i,\dots,9$$

**Step 2:** Compute swap annuity:

$$A_i^{(m)}(T_i) = \sum_{k=i+1}^{10} D_k^{(m)}(T_i)$$

**Step 3:** Compute par swap rate:

$$y_i^{(m)}(T_i) = \frac{1 - D_{10}^{(m)}(T_i)}{A_i^{(m)}(T_i)}$$

**Step 4:** Payer swaption payoff:

$$\text{Payoff}_{i,K}^{(m)}(T_i) = L \cdot A_i^{(m)}(T_i) \cdot \max\big(y_i^{(m)}(T_i) - K, 0\big)$$

### 3. Discounting to $t=0$ with Numéraire

Under $\mathbb{Q}^{10}$ with numéraire $D_{10}(t)$:

$$V_{0,i,K}^{(m)} = \text{Payoff}_{i,K}^{(m)}(T_i) \cdot \frac{D_{10}(0)}{D_{10}^{(m)}(T_i)}$$

**Monte Carlo estimator:**

$$\widehat{V}_{0,i,K}^{\text{MC}} = \frac{1}{M} \sum_{m=1}^{M} V_{0,i,K}^{(m)}$$

$$\text{SE}_{i,K} = \frac{\text{Std}(V_{0,i,K}^{(m)})}{\sqrt{M}}$$

### 4. Z-score Comparison

$$Z_{i,K} = \frac{\widehat{V}_{0,i,K}^{\text{MC}} - V_{0,i,K}^{\text{Gauss}}}{\text{SE}_{i,K}}$$

where $V_{0,i,K}^{\text{Gauss}}$ is from the Gaussian closed-form formula.

**Interpretation:** $|Z_{i,K}| \leq 2$ indicates no statistically significant difference at the 5% level.

## Monte Carlo Swaption Pricing Results

**Simulation Parameters:**
- Number of paths: $M = 100{,}000$
- Time step: $\Delta t = 0.01$
- Volatility: $\sigma = 0.01$
- Notional: $L = 100$

### Comparison Table: MC vs Gaussian Prices with Z-scores

| Maturity | Strike | MC Price | Gaussian Price | Std Error | Z-score |
|----------|--------|----------|----------------|-----------|---------|
| **T₁** | 1% | 15.1843 | 15.1828 | 0.0255 | 0.06 |
| | 2% | 8.1923 | 8.1891 | 0.0220 | 0.14 |
| | 3% | 3.0151 | 3.0157 | 0.0147 | -0.04 |
| | 4% | 0.6306 | 0.6298 | 0.0066 | 0.12 |
| | 5% | 0.0662 | 0.0642 | 0.0019 | 1.02 |
| **T₂** | 1% | 13.5957 | 13.5660 | 0.0298 | 1.00 |
| | 2% | 7.9505 | 7.9377 | 0.0252 | 0.51 |
| | 3% | 3.7275 | 3.7331 | 0.0183 | -0.31 |
| | 4% | 1.3165 | 1.3210 | 0.0108 | -0.41 |
| | 5% | 0.3269 | 0.3325 | 0.0051 | -1.11 |
| **T₃** | 1% | 12.0423 | 12.0107 | 0.0302 | 1.05 |
| | 2% | 7.4490 | 7.4295 | 0.0256 | 0.76 |
| | 3% | 3.9489 | 3.9397 | 0.0194 | 0.47 |
| | 4% | 1.7323 | 1.7279 | 0.0128 | 0.34 |
| | 5% | 0.6067 | 0.6075 | 0.0074 | -0.11 |
| **T₄** | 1% | 10.4291 | 10.4282 | 0.0283 | 0.03 |
| | 2% | 6.7120 | 6.7171 | 0.0241 | -0.21 |
| | 3% | 3.8390 | 3.8403 | 0.0188 | -0.07 |
| | 4% | 1.9031 | 1.9040 | 0.0133 | -0.07 |
| | 5% | 0.8008 | 0.8020 | 0.0084 | -0.15 |
| **T₅** | 1% | 8.8026 | 8.7973 | 0.0251 | 0.21 |
| | 2% | 5.8479 | 5.8460 | 0.0215 | 0.09 |
| | 3% | 3.5244 | 3.5241 | 0.0172 | 0.02 |
| | 4% | 1.8930 | 1.8955 | 0.0127 | -0.20 |
| | 5% | 0.8936 | 0.8963 | 0.0086 | -0.31 |
| **T₆** | 1% | 7.1087 | 7.1163 | 0.0209 | -0.36 |
| | 2% | 4.8393 | 4.8486 | 0.0181 | -0.51 |
| | 3% | 3.0329 | 3.0421 | 0.0147 | -0.63 |
| | 4% | 1.7297 | 1.7356 | 0.0111 | -0.53 |
| | 5% | 0.8862 | 0.8903 | 0.0079 | -0.52 |
| **T₇** | 1% | 5.3725 | 5.3898 | 0.0162 | -1.08 |
| | 2% | 3.7332 | 3.7489 | 0.0140 | -1.12 |
| | 3% | 2.4160 | 2.4276 | 0.0115 | -1.00 |
| | 4% | 1.4419 | 1.4490 | 0.0090 | -0.79 |
| | 5% | 0.7857 | 0.7900 | 0.0065 | -0.67 |
| **T₈** | 1% | 3.6071 | 3.6241 | 0.0110 | -1.55 |
| | 2% | 2.5497 | 2.5651 | 0.0096 | -1.61 |
| | 3% | 1.6931 | 1.7044 | 0.0080 | -1.42 |
| | 4% | 1.0476 | 1.0546 | 0.0063 | -1.12 |
| | 5% | 0.5988 | 0.6031 | 0.0047 | -0.91 |
| **T₉** | 1% | 1.8197 | 1.8255 | 0.0056 | -1.06 |
| | 2% | 1.3058 | 1.3116 | 0.0049 | -1.18 |
| | 3% | 0.8856 | 0.8906 | 0.0041 | -1.21 |
| | 4% | 0.5639 | 0.5675 | 0.0033 | -1.09 |
| | 5% | 0.3350 | 0.3373 | 0.0025 | -0.93 |

### Statistical Analysis

**Z-score Interpretation:** The Z-score measures the standardized difference between Monte Carlo and Gaussian prices:

$$Z_{i,K} = \frac{\widehat{V}_{0,i,K}^{\text{MC}} - V_{0,i,K}^{\text{Gauss}}}{\text{SE}_{i,K}}$$

**Key Findings:**

1. **All Z-scores are within [-2, 2]**: This indicates **no statistically significant difference** between the full LMM Monte Carlo prices and the Gaussian closed-form approximation at the 5% significance level.

2. **Gaussian formula is highly accurate**: The Gaussian swaption pricing formula provides an excellent approximation to the true LMM swaption values across all maturities ($T_1, \ldots, T_9$) and strikes (1% to 5%).

3. **Pattern by maturity**: 
   - Short maturities ($T_1$-$T_3$): Z-scores very close to zero, indicating near-perfect agreement
   - Longer maturities ($T_7$-$T_9$): Slightly larger negative Z-scores (around -1 to -1.6), but still within acceptable range

4. **Pattern by strike**: All strikes show good agreement with the Gaussian approximation, with no systematic bias.

**Conclusion:** The Gaussian approximation is valid for pricing payer swaptions in this Gaussian LIBOR Market Model setup. The differences between MC and analytical prices are purely due to Monte Carlo sampling error, not model mis-specification.